# Part 1: n-gram Language Modeling

In this part of the assignment, we'll expand on the `SimpleTrigramLM` from the live session demo. We'll add smoothing to improve performance on unseen data, and explore some of the properties of the smoothed model.

If you haven't looked over the simple trigram LM in [lm1.ipynb](../../../materials/simple_lm/lm1.ipynb), we recommend familiarizing yourself with it. This assignment will use a very similar set-up.

## A note on notation

"Primed" variables, e.g. $c′$. This is just a dummy variable, not necessarily equal to $c$ but usually assumed to be from the same domain. For example: $P(c)$ is the probability of a particular word $c$, while $\sum_{c′} P(c′)$ is the sum of the probabilities of all possible words $c′$. If not explicitly stated, the domain is usually the domain of the expression inside the sum - so in this case we'd sum for all words $c′ \in V$ in the vocabulary.
 
- Set notation: $\{x : f(x) > 0\}$ means the set of all $x$ where $f(x)>0$. Strictly speaking, we should write $\{x∈S:f(x)>0\}$ where $S$ is some other set, but often we omit this when $S$ is implied (such as words (types) in the vocabulary). So $\{ b′:C_{b′c} > 0 \}$ is the set of all words (types) $b′$ where the counts $C_{b′c}$ (for some particular word $c$) are greater than zero.
 
- Similar to the above, $\left| x:f(x)>0 \right|$ means the number of elements (size) of that set. So $\left| b′:C_{b′c} > 0 \right|$ is the number of words $b′$ where the counts $C_{b′c}$ (for some particular word $c$) are greater than zero.

In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats

import nltk

# Helper libraries for this notebook
from w266_common import utils, vocabulary
import ngram_lm, ngram_lm_test

# Add-k Smoothing

Recall our unsmoothed maximum likelihood estimate of $ P(w_i\ |\ w_{i-1}, w_{i-2})$ where we use the raw distribution over words seen in a context in the training data:

$$  \hat{P}(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc}}{\sum_{c'} C_{abc'}} $$

Add-k smoothing is the simple refinement where we add $k > 0$ to each count $C_{abc}$, pretending we've seen every vocabulary word $k$ extra times in each context. So we have:

$$ \hat{P}_k(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc} + k}{\sum_{c'} (C_{abc'} + k)} = \frac{C_{abc} + k}{C_{ab} + k\cdot|V|} $$

where $|V|$ is the size of our vocabulary.

In the questions below and in the code, we'll refer to $(w_{i-2}, w_{i-1})$ as the *context*, and $w_i$ as the current *word*. By convention, we'll somewhat interchangeably refer to the sequence $(w_{i-2}, w_{i-1}, w)$ as $abc$.

### Part (a): Short answer questions

Give brief answers to the following, in the cell below.

1. If we encounter a new context `(foo, bar)` unseen in the training data, what will the predicted *distribution* $\hat{P}_k(w\ |\ \text{foo}, \text{bar})$ be? How does your answer depend on $k$?
<p>
2. Is this a good estimate, or can we do better?  
<p><p>
3. If we encounter a new word in a familiar context (i.e. `ab` is in the corpus, but `abq` is not), what will our predicted probability $\hat{P}_k(q\ |\ b, a)$ be? Give your answer in terms of $C_{ab}$, $k$, and $|V|$.
<p>
4. Based on your answer to question 3, in which context will your model predict a higher probability of *any* unknown word?  
Context (a): `<s> the ___`  
Context (b): `Mister Rogers ___`  
Assume $C_{\text{<s>, the}} = 10000$ and $C_{\text{Mister, Rogers}} = 47$.
<p>
5. Based on your knowledge of language, which of the contexts from question 4 *should* have a higher probability of an unknown word?

### Answers for Part (a)

Please keep answers brief (1-2 lines).

Hint: You can use LaTeX to typeset math, e.g. `$ f(x) = x^2 $` will render as $ f(x) = x^2 $.

1. $\frac{1}{|V|}$. It doesn't depend on k if we have never seen the context before.
2. It predicts an even distribution for all words. Not a good estimate because some words such as "Francisco" won't follow (foo, bar). Differences in word fertility matters should be accounted for.
3. $\frac{k}{c_{ab} + k|V|}$
4. Context b would predict a higher probability for any unknown word. This gives a lower denominator and higher probability overall.
5. Context a should predict a higher probability. (<s>, the) is more generic and fertile.

## Part (b): Implementing the Add-k Model

Despite its shortcomings, it's worth implementing an add-k model as a baseline. Unlike the unsmoothed model, we'll be able to get some reasonable (or at least, finite) perplexity numbers which we can compare to the Kneser-Ney model below.

We've provided some skeleton code (similar to [lm1.ipynb](../../../materials/simple_lm/lm1.ipynb)) in the `ngram_lm.py` file. In the `AddKTrigramLM` class, implement the following:
- `__init__(self, words)`, which computes the necessary corpus statistics $C_{abc}$ and $C_{ab}$.
- `next_word_proba(self, word, seq, k)`, which computes $\hat{P}_k(w\ |\ w_{i-1}, w_{i-2})$

See the function docstrings and in-line comments for more details. In particular, you may want to use `collections.defaultdict` and `dict.get()` to simplify handling of unknown keys. See [dict_notes.md](dict_notes.md) for a brief overview of how to use these.

**Note on keys and word-order:** Convention in the math is to write context in reverse order, as in $P(w\ |\ w_{i-1}, w_{i-2})$, but in the code it'll be much easier to write things left-to-right as in normal English: so for the context "`foo bar ___`", you'll want to use `("foo", "bar")` as a dict key.



In [2]:
reload(ngram_lm)
utils.run_tests(ngram_lm_test, ["TestAddKTrigramLM"])

test_context_totals (ngram_lm_test.TestAddKTrigramLM) ... ok
test_counts (ngram_lm_test.TestAddKTrigramLM) ... ok
test_next_word_proba_k_exists (ngram_lm_test.TestAddKTrigramLM) ... ok
test_next_word_proba_no_smoothing (ngram_lm_test.TestAddKTrigramLM) ... ok
test_no_mutate_on_predict (ngram_lm_test.TestAddKTrigramLM) ... ok
test_words (ngram_lm_test.TestAddKTrigramLM) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.014s

OK


# Kneser-Ney Smoothing

In this part, we'll explore Kneser-Ney smoothing as a more sophisticated way of estimating unseen probabilities. 

When building an n-gram model, we're limited by the model order (e.g. trigram, 4-gram, or 5-gram) and how much data is available. Within that, we want to use as much information as possible. Within, say, a trigram context, we can compute a number of different statistics that might be helpful. Let's review a few goals:
1. If we don't have good n-gram estimates, we want to back off to (n-1) grams.
2. If we back off to (n-1) grams, we should do it "smoothly".
3. Our counts $C_{abc}$ are probably _overestimates_ for the n-grams we observe (see *held-out reweighting*).
4. Type fertilities tell us more about $P(w_{new}\ |\ \text{context})$ than the unigram distribution does.

Kneser-Ney smoothing combines all four of these ideas. 

**Absolute discounting** - which follows from 3. - gives us an easy way to backoff (1. and 2.), by distributing the subtracted probability mass among the backoff distribution $\tilde{P}(c\ |\ b)$.  The amount to redistribute, $\delta$, is a hyperparameter selected based on a cross-validation set in the usual way, although for this assignment we'll just let $\delta = 0.75$.

$$ P_{ad}(c\ |\ b, a) = \frac{C_{abc} - \delta}{C_{ab}} + \alpha_{ab} \tilde{P}(c\ |\  b) $$

Where $\alpha_{ab}$ is a backoff factor, derived from the counts, that guarantees that the probabilities are normalized: $\sum_{c'} P_{ad}(c'\ |\ b, a) = 1$. This definition is recursive: if we let $\tilde{P}(c\ |\  b) = P_{ad}(c\ |\ b)$, then the backoff distribution can also back off to even lower n-grams.

*Note:* we need the numerator above to positive, so it should actually read $\max(0, C_{abc} - \delta)$.

**Type fertility** is item 4. Instead of falling back to the unigram distribution at the end, we'll define $\hat{P}(w)$ as proportional to the type fertility of $w$, or the *number of unique preceding words* $w_{i-1}$.  In the following equation, the word we are estimating the probability of is $c$.  $b'$ are the set of words we've found occurring before $c$ in the training data.

$$ \hat{P}_{tf}(c) \propto \left|\ b' : C_{b'c} > 0\ \right| = tf(c)$$

In order to make this a valid probability distribution, we need to normalize it with a factor $Z_{tf} = \sum_{w} tf(w)$, so we have $\hat{P}_{tf}(w) = \frac{tf(w)}{Z_{tf}} $

### KN Equations

Putting it all together, we have our equations for a KN trigram model:

$$ P_{kn}(c\ |\ b, a) = \frac{\max(0, C_{abc} - \delta)}{C_{ab}} + \alpha_{ab} P_{kn}(c\ |\  b) $$
where the bigram backoff is:
$$ P_{kn}(c\ |\ b) = \frac{\max(0, C_{bc} - \delta)}{C_{b}} + \alpha_{b} P_{kn}(c) $$
and the unigram (type fertility) backoff is:
$$ P_{kn}(c) = \frac{tf(c)}{Z_{tf}} \quad \text{where} \quad tf(c) = \left|\ b' : C_{b'c} > 0\ \right| $$

Note that there is only one free parameter in in the above equations: $\delta$. You'll compute $\alpha_{ab}$ and $\alpha_{b}$ in the exercise below.

### Part (c): Short answer questions

Give brief answers to the following, in the cell below.

1. Compute the value of $\alpha_b$ such that $P_{kn}(c\ |\ b)$ is properly normalized. Give your answer in terms of the discount term $\delta$, the context total $C_{b}$, and $\text{nnz}(b) = \left|\ c' : C_{bc'} - \delta > 0 \ \right|$, the number of bigrams `bc'` with positive (discounted) count.  
**Hint:** solve $\sum_{c'} P_{kn}(c'\ |\ b) = 1$ using the fact that $\sum_{c'} P_{kn}(c') = 1$ and $\sum_{c'} C_{bc'} = C_{b} $.  
Note that if you replace $b$ with the context $ab$, you can re-use this result to define $\alpha_{ab}$.  
<p>
2. Based on your answer to question 1, in which case do you expect the KN model to rely more on the backoff distribution (i.e. higher $\alpha$)? Assume $\delta = 0.75$.   
Case (a): context `<s> my name is ___`, which occurs 1000 times ending with 632 unique words.  
Case (b): context `Mister Rogers ___`, which occurs 5 times and always ends with "Neighborhood".  
Explain _briefly_ why this is the case, and why this is reasonable behavior given your intuition.

### Answers for Part (c)

You can use LaTeX to typeset math, e.g. `$ f(x) = x^2 $` will render as $ f(x) = x^2 $. For Question 1, **please show your work!** You can use LaTeX, or attach an image to your submission. To add an image to the notebook, use Markdown: `![alt_text](partc1.png)`

1. ![alt_text](partc1.png.jpg)
2. $\alpha$ determines how much the probability depends on backoff distribution. Holding $\delta$ constant, in case a) $nnz(ab)/C_{ab}=0.632$ and in case b) $nnz(ab)/C_{ab}=0.2$. $\alpha$ is higher in case a) and reasonable because the possible trigram in case a) are rarer, on average each `my name is ___` trigram occurs less than twice. The trigram in case b) is less rare, the same phrase occurred 5 times. Therefore, case a) needs to rely on the backoff distribution more.

## Part (d): Implementing the KN Model

Implement the `KNTrigramLM` in `ngram_lm.py`. As with the add-k model, we've provided some starter code for you; you need only fill in the marked blocks. `KNTrigramLM` also conforms to the exact same interface as the add-k model.

You should:
- Finish the implementation of `__init__(self, words)` to compute the necessary quantities
- Implement the `kn_interp(...)` function, which interpolates between n-gram counts and a backoff distribution according to the absolute discounting rule (see definitions of $P_{kn}(c\ |\ a, b)$ and $P_{kn}(c\ |\ b)$). You'll need your definition of $\alpha$ from (c).1. here.

As before, see the function docstrings and in-line comments for more details.

When you're done implementing it, run the cells below to train your model, sample some sentences, and evaluate your model on the dev set. Then jump to Part (e) for a few last questions.

In [3]:
reload(ngram_lm)
utils.run_tests(ngram_lm_test, ["TestKNTrigramLM"])

test_context_nnz (ngram_lm_test.TestKNTrigramLM) ... ok
test_context_totals (ngram_lm_test.TestKNTrigramLM) ... ok
test_counts (ngram_lm_test.TestKNTrigramLM) ... ok
test_kn_interp (ngram_lm_test.TestKNTrigramLM) ... ok
test_next_word_proba (ngram_lm_test.TestKNTrigramLM) ... ok
test_no_mutate_on_predict (ngram_lm_test.TestKNTrigramLM)
Don't allow modifications to the LM during inference. ... ok
test_type_contexts (ngram_lm_test.TestKNTrigramLM) ... ok
test_type_fertility (ngram_lm_test.TestKNTrigramLM) ... ok
test_words (ngram_lm_test.TestKNTrigramLM) ... ok
test_z_tf (ngram_lm_test.TestKNTrigramLM) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.013s

OK


# Training your Model

The same code below can be used with either model; in the cell where it says "Select your Model", you can choose the add-k model or the KN model.

## Loading & Preprocessing
Once again, we'll build our model on the Brown corpus. We'll do an 80/20 train/test split, and preprocess words by lowercasing and replacing digits with `DG` (so `2016` becomes `DGDGDGDG`).

In a slight departure from the `lm1.ipynb` demo, we'll restrict the vocabulary to 40000 words. This way, a small fraction of the *training* data will be mapped to `<unk>` tokens, and the model can learn n-gram probabilities that include `<unk>` for prediction on the test set. (If we interpret `<unk>` as meaning "rare word", then this is somewhat plausible as a way to infer things about the class of rare words.)

In [4]:
assert(nltk.download('brown'))  # Make sure we have the data.
corpus = nltk.corpus.brown
V = 30000
train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=False)
# Build vocabulary only on the training set.
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(train_sents)), size=V)
print("Train set vocabulary: {:,} words".format(vocab.size))

[nltk_data] Downloading package brown to /home/yeunghoman/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Loaded 57,340 sentences (1.16119e+06 tokens)
Training set: 45,872 sentences (979,646 tokens)
Test set: 11,468 sentences (181,546 tokens)
Train set vocabulary: 30,000 words


Our smoothed models will also be trigram models, so for convenience we'll also prepend *two* `<s>` markers. (We could avoid this, but then we'd need special handling for the first token of each sentence.)

To make it easier to work with, we'll take the list of tokens as a NumPy array.

In [5]:
def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = ([u"<s>", u"<s>"] + s + [u"</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in utils.flatten(padded_sentences)], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)
print("Sample data: \n", repr(train_tokens[:20]))

Sample data: 
 array(['<s>', '<s>', 'the', 'fulton', 'county', 'grand', 'jury', 'said',
       'friday', 'an', 'investigation', 'of', "atlanta's", 'recent',
       'primary', 'election', 'produced', '``', 'no', 'evidence'], dtype=object)


In [6]:
print("Sample data: \n", repr(train_tokens[-20:]))

Sample data: 
 array(['and', 'it', 'was', 'because', 'of', 'an', 'old', 'norberg',
       'inheritance', 'that', 'i', 'got', 'to', 'understand', 'them',
       'all', 'so', 'well', '.', '</s>'], dtype=object)


In [7]:
print("Sample data: \n", repr(train_tokens[-50:]))

Sample data: 
 array(['among', 'them', ',', 'with', 'blonde', 'hair', 'and', 'pert',
       'faces', '.', '</s>', '<s>', '<s>', 'handley', 'married', 'freya',
       'and', 'red', ',', 'of', 'the', 'red', 'beard', ',', 'married',
       'anta', '.', '</s>', '<s>', '<s>', 'and', 'it', 'was', 'because',
       'of', 'an', 'old', 'norberg', 'inheritance', 'that', 'i', 'got',
       'to', 'understand', 'them', 'all', 'so', 'well', '.', '</s>'], dtype=object)


## Select your model

Select either `AddKTrigramLM` or `KNTrigramLM` in the cell below. If switching models, you only need to re-run the cells below here - no need to re-run the preprocessing.

In [8]:
reload(ngram_lm)

# Uncomment the line below for the model you want to run.
# Model = ngram_lm.AddKTrigramLM
Model_KN = ngram_lm.KNTrigramLM

t0 = time.time()
print("Building trigram LM... ", end="")
lm_KN = Model_KN(train_tokens)
print("done in {:.02f} s".format(time.time() - t0))
lm_KN.print_stats()

Building trigram LM... done in 5.59 s
=== N-gram Language Model stats ===
  30,000 unique 1-grams
 357,896 unique 2-grams
 732,467 unique 3-grams
Optimal memory usage (counts only): 24.21 MB


In [9]:
reload(ngram_lm)

# Uncomment the line below for the model you want to run.
Model_AddK = ngram_lm.AddKTrigramLM
#Model = ngram_lm.KNTrigramLM

t0 = time.time()
print("Building trigram LM... ", end="")
lm_AddK = Model_AddK(train_tokens)
print("done in {:.02f} s".format(time.time() - t0))
lm_AddK.print_stats()

Building trigram LM... done in 3.34 s
=== N-gram Language Model stats ===
       0 unique 1-grams
       0 unique 2-grams
 732,467 unique 3-grams
Optimal memory usage (counts only): 16.70 MB


Change `params` to change the smoothing factor. `AddKTrigramLM` will ignore the value of `delta`, and `KNTrigramLM` will ignore `k`.

In [10]:
lm_KN.set_live_params(k = 0.001, delta=0.75)
lm_AddK.set_live_params(k = 0.001, delta=0.75)
# lm.set_live_params(k = 0.01, delta=0.25)

## Sampling Sentences

In [11]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(lm_KN.sample_next(seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print(" ".join(seq))
    print("[{1:d} tokens; log P(seq): {0:.02f}]\n".format(*lm_KN.score_seq(seq)))

<s> <s> the most off told loss of england , had stated that any other authority and are to comprehend also mean
[20 tokens; log P(seq): -155.40]

<s> <s> there is perhaps one or two empires eventually DG/DG'' '' thick . </s>
[12 tokens; log P(seq): -88.76]

<s> <s> some of the american could deft , hilarious picture is use <unk> rooms of understanding . </s>
[16 tokens; log P(seq): -111.81]

<s> <s> how long has been arrested -- DGDGDG , DGDGDG ) , as in other forms of low anionic binding capacity
[20 tokens; log P(seq): -94.30]

<s> <s> not all the as he left sean treats stance , one person in a person strolling with lullaby bond generally
[20 tokens; log P(seq): -174.52]



In [12]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(lm_AddK.sample_next(seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print(" ".join(seq))
    print("[{1:d} tokens; log P(seq): {0:.02f}]\n".format(*lm_AddK.score_seq(seq)))

<s> <s> they practiced communications swadesh want alice mendelssohn pfc. begun friends associated blue liter rupture archaic istanbul god-like unpaired counter conic
[20 tokens; log P(seq): -283.30]

<s> <s> for radiopasteurization desegregation adler spying self-centered huntley gram proprietors threw healer proclaim lyricist default canvassers suffrage indians dines vaults sauerkraut
[20 tokens; log P(seq): -283.41]

<s> <s> fear villa several boos intermission accommodation something haflis hell sweep orwell's child's hinted nucleoli plastered pies huntington rpm exceeding miglia
[20 tokens; log P(seq): -297.16]

<s> <s> it is asking priming goitre lends small-game shred revelations authentic bathtub joy pro-yankee excerpts ld questioner overgrown unsurpassed binders portfolio-maker
[20 tokens; log P(seq): -269.83]

<s> <s> still existing paramilitary crazily sequoia overflowing impressionistic spontaneous enthusiasts proxy bi suitably explanation promises abduction plucked adminis

## Scoring on Held-Out Data

Your KN model should get a perplexity of around 280 on the dev set with $\delta = 0.75$. The add-k smoothing model will perform... somewhat worse :)

In [12]:
log_p_data, num_real_tokens = lm_KN.score_seq(train_tokens)
print("Train perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

log_p_data, num_real_tokens = lm_KN.score_seq(test_tokens)
print("Test perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

Train perplexity: 17.14
Test perplexity: 283.93


In [13]:
log_p_data, num_real_tokens = lm_AddK.score_seq(train_tokens)
print("Train perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

log_p_data, num_real_tokens = lm_AddK.score_seq(test_tokens)
print("Test perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

Train perplexity: 43.05
Test perplexity: 3599.52


## Part (e): Additional Questions

Answer the following in the cell below.

1. What is the average number of times our model sees any particular trigram, averaged across the trigrams we observed at least once (i.e. ignoring the zeros for trigrams we never observed)? How about averaged across *all possible* trigrams (i.e. including hypothetical ones we never observed)? (*Hint:* you don't need to write any code for this - it should be a quick calculation.)
<p>
2. Based on your answer above, do you think that a 4-gram or 5-gram model would perform better than the trigram model on the 1 million word Brown corpus? How about on a 42-billion word Wikipedia corpus?  
<p><p>
3. Which model generates more "realistic" sentences - `AddKTrigramLM`, `KNTrigramLM`, or the unsmoothed `SimpleTrigramLM` from the demo notebook? Is this in-line with their perplexity on the dev set?

*Note:* in the next assignment, we'll implement a neural network model that avoids the sparsity problem altogether and can achieve significantly better generalization even on a small dataset like the Brown corpus.

### Answers for Part (e)

1.  The number of trigrams the model should ideally observe is `(979,646 tokens - 45,872sent*2offsets)`, the number of unique trigrams the model observed is roughly 732,467. So the first answer should be `(979646-45872*2)/732,467 = 1.212208`. Hypothetically, we have `|V|^3=30,000^3` possible 3-grams. The second answer should be `(979646-45872*2)/(30,000^3) = 3.288526e-08`.

2. Holding corpus size at 1M and raiseing `N`-gram from `N=3` to `N=4`, the average number of times our model sees any particular N-gram across all possible `N`-grams (let's call this `#seen/#possible`) decreases by a division of `|V|=30,000^1` (and `|V|=30,000^2` for `5`-gram). Given the `#seen/#possible=3.288526e-08` is already very small with `3`-gram, it will only go extremely small with `4`-gram (`1.0961e-12`) or `5`-gram (`3.653e-17`). In a sense we are creating more features and data representation became a lot sparser. Thus `4` or `5`-gram will not do better than `3`-gram model. On the other hand, increasing corpus size to `42B`, holding `|V|=30,000` and raising N-gram from `3` to `4` may actually make the model perform better. Here `#seen/#possible=5.185e-08` decreases by a division of `|V|=30,000` but also increases by a multiple of `42,000` because of increased sample size. The increased predictive power of `4`-gram and lower data-sparsity will actually increase model performance. Lastly, using `42B` word corpus and raising to `5`-gram will yield a lower `#seen/#possible=1.728e-12` compared to `1M` word corpus with `3`-gram model `#seen/#possible=3.628e-08`, here it's not clear whether the increased predictive power of 5-gram will actually offset the increaed sparsity,

3. The SimpleTrigramModel generated the most readable sentences, followed by the KN model and the worst is the AddK model. This is because the SimpleTrigramModel is trained to over-fit and only memorize the training trigrams exactly as they appear in the training data, while the KN and AddK model uses smoothing to model phrases that may statistically exist but not necessarily exist semantically. As such, the generated sentences from the SimpleTrigramModel resembles the fluent sentences found from the training data a lot more, while the other two models generated nonsensical sentences quite often. Comparing the KN and AddK models, the KN model uses more sophisticated smoothing techniques such as considering type fertility therefore generated more realistic sentences than the AddK model. This ordering is actually not completely aligned with the development set perplexities. The SimpleTrigramModel easily generate infinity as perplexity for any unseen tri-grams in the development dataset, the AddK model generate some what high perplexity due to its unsophisticated smoothing techniques and the KN model generated the lowest perplexity because its recursive back-off mechanism and type fertility accounting are more robust to unseen trigrams in development set. Note that the SimpleTrigramModel in the demo notebook also used a larger training vocabulary (43707 words) so the comparisons are not strictly apple to apple.

In [2]:
print ((979646-45872*2)/(30000**3))
print ((979646-45872*2)/(30000**4))
print ((979646-45872*2)/(30000**5))
print (42000000000/(30000**3))
print (42000000000/(30000**4))
print (42000000000/(30000**5))

3.288525925925926e-08
1.0961753086419753e-12
3.653917695473251e-17
0.0015555555555555555
5.185185185185185e-08
1.7283950617283951e-12


## Just for fun: Linguistic Curiosities

You might have seen this floating around the internet:
![Adjective Order](adjective_order.jpg)
*source: https://twitter.com/MattAndersonBBC/status/772002757222002688?lang=en*

Let's see if it holds true, statistically at least. Note that log probabilities are always negative, so the smaller magnitude is better. And remember the log scale: a difference of score of 8 units means one utterance is $2^8 = 256$ times more likely!

In [16]:
def preprocess_for_scoring(sentence, vocab):
    # Pre-process words, replace anything the model doesn't know with <unk>
    words = [utils.canonicalize_word(w, wordset=vocab.wordset)
             for w in sentence]
    # Pad sequence with start and end markers
    return [u"<s>", u"<s>"] + words + [u"</s>"]

s0 = preprocess_for_scoring("square green plastic toys".split(), vocab)
s1 = preprocess_for_scoring("plastic green square toys".split(), vocab)

In [17]:
print("s0 score: {:.02f}".format(lm_KN.score_seq(s0)[0]))
print("s1 score: {:.02f}".format(lm_KN.score_seq(s1)[0]))

s0 score: -52.01
s1 score: -60.96


In [18]:
print("s0 score: {:.02f}".format(lm_AddK.score_seq(s0)[0]))
print("s1 score: {:.02f}".format(lm_AddK.score_seq(s1)[0]))

s0 score: -59.20
s1 score: -59.20


In [19]:
noun = "toys"
adjectives = ["square", "green", "plastic"]
results = []
for adjs in itertools.permutations(adjectives):
    words = list(adjs) + [noun]
    seq = preprocess_for_scoring(words, vocab)
    score, _ = lm_KN.score_seq(seq)
    results.append((score, words))

# Sort results
for score, words in sorted(results, reverse=True):
    print("\"{:s}\" : {:.02f}".format(" ".join(words), score))

"green square plastic toys" : -51.95
"square green plastic toys" : -52.01
"plastic square green toys" : -60.96
"plastic green square toys" : -60.96
"green plastic square toys" : -61.25
"square plastic green toys" : -61.31


In [20]:
noun = "toys"
adjectives = ["square", "green", "plastic"]
results = []
for adjs in itertools.permutations(adjectives):
    words = list(adjs) + [noun]
    seq = preprocess_for_scoring(words, vocab)
    score, _ = lm_AddK.score_seq(seq)
    results.append((score, words))

# Sort results
for score, words in sorted(results, reverse=True):
    print("\"{:s}\" : {:.02f}".format(" ".join(words), score))

"square plastic green toys" : -59.20
"square green plastic toys" : -59.20
"plastic square green toys" : -59.20
"plastic green square toys" : -59.20
"green square plastic toys" : -59.20
"green plastic square toys" : -59.20
